In [0]:
!pip install sklearn_crfsuite

#Significado das abreviações da NLTK
#https://medium.com/@gianpaul.r/tokenization-and-parts-of-speech-pos-tagging-in-pythons-nltk-library-2d30f70af13b

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# **Dataset Twitter**



## Geração e Organização do Dataframe


In [0]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lgRanN-EIOqgW1LJMhOJu2S3x29p_3Cs' -O tweets.txt

--2019-11-13 19:17:01--  https://docs.google.com/uc?export=download&id=1lgRanN-EIOqgW1LJMhOJu2S3x29p_3Cs
Resolving docs.google.com (docs.google.com)... 74.125.142.139, 74.125.142.138, 74.125.142.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lamqmu3hfdlp8bnbmslsm0he31s6oo38/1573668000000/17742136402446089806/*/1lgRanN-EIOqgW1LJMhOJu2S3x29p_3Cs?e=download [following]
--2019-11-13 19:17:02--  https://doc-0o-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lamqmu3hfdlp8bnbmslsm0he31s6oo38/1573668000000/17742136402446089806/*/1lgRanN-EIOqgW1LJMhOJu2S3x29p_3Cs?e=download
Resolving doc-0o-9o-docs.googleusercontent.com (doc-0o-9o-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to doc-0o-9o-docs.googleusercontent.com (doc-0o-9o-do

In [0]:
#Reading the csv file
df_tweet = pd.read_csv('tweets.txt', delimiter="Tweet fetched:", encoding = "UTF-8")
df_tweet.columns = ["Sentence #","Word"]
#df["POS"] = np.nan
#df["Tag"] = np.nan
#Display first 10 rows
df_tweet.head(10)
print(df_tweet.values)

[['@ForeverPlayerG tem que ser , essa o áudio tava ruim . Próxima faz a farm de iron na live.'
  nan]
 ['Exception' nan]
 [nan nan]
 ...
 ['@flaviofachel Somos 7 milhões nesse momento!' nan]
 [nan nan]
 ['queria entender oq se passa pela cabeça das pessoas' nan]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df_tweet = df_tweet[df_tweet["Sentence #"] != "Exception"]
df_tweet = df_tweet.dropna(subset=['Sentence #'])
df_tweet = df_tweet.reset_index()
df_tweet = df_tweet.loc[:,~df_tweet.columns.isin(['index'])]
df_tweet.head(10)


,Sentence #,Word
0,"@ForeverPlayerG tem que ser , essa o áudio tav...",NaN
1,esse João Guilherme ta mt estranho,NaN
2,não quero ir pra escola,NaN
3,@AbioJoao Patrão,NaN
4,Preciso acordar mais cedo e ajeitar meu cabelo...,NaN
5,#depoisdojogo Piperno se o Trainee R.Ceni pass...,NaN
6,Tenho Msm irmão kkkkk https://t.co/9YkN6SehqG,NaN
7,Quero conversar com alguém mano,NaN
8,@belaapacheco meu sonho eh te pegar,NaN
9,Mas fica em paz! Eu te amo!,NaN


In [0]:
df_tweet["Sentence #"][0]

'@ForeverPlayerG tem que ser , essa o áudio tava ruim . Próxima faz a farm de iron na live.'

In [0]:
!pip3 install spacy
!python -m spacy download pt
!python -m spacy download en
import spacy
nlp = spacy.load('pt')

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


## Tokenização

In [0]:
def tokenize_df(data):
  from nltk.chunk import conlltags2tree, tree2conlltags
  from pprint import pprint
  #from nltk.tokenize import TweetTokenizer
  #tknzr = TweetTokenizer()
  import nltk
  df_temp = pd.DataFrame()
  dict_temp = {}
  for i in range(len(data)):
    text = nltk.word_tokenize(data['Sentence #'][i])
    s = nltk.pos_tag(text)

    for j in range(len(s)):
      dict_temp["Sentence #"] = "Sentence: {}".format(i)
      dict_temp["Word"] = s[j][0]
      dict_temp["POS"] = s[j][1]
      #dict_temp["Tag"] = s[j][2]
      dict_temp["Tag"] = [(token.orth_, token.pos_) for token in nlp(u'{}'.format(text[j]))][0][1]
      #print(s[j][0],'-',dict_temp["Tag"])
      df_temp = df_temp.append(dict_temp, ignore_index=True)
    
    print("Sentença {} tokenizada.\n".format(i))
  
  col = df_temp['Word']
  df_temp.drop(labels=['Word'], axis=1,inplace = True)
  df_temp.insert(0, 'Word', col)
  col = df_temp['Sentence #']
  df_temp.drop(labels=['Sentence #'], axis=1,inplace = True)
  df_temp.insert(0, 'Sentence #', col)
  data = df_temp
  return data
#del df_temp


In [0]:
print(type(df_tweet))
df_tweet = tokenize_df(df_tweet)
df_tweet

In [0]:
df_tweet.describe(include = 'all')

,Sentence #,Word,POS,Tag
count,18023,18023,18023,18023
unique,1612,5768,39,14
top,Sentence: 831,@,NN,NOUN
freq,34,558,7940,4924


In [0]:
#Displaying the unique Tags
df_tweet['Tag'].unique()

array(['NOUN', 'VERB', 'SCONJ', 'PUNCT', 'DET', 'ADP', 'ADV', 'PRON',
       'PROPN', 'ADJ', 'NUM', 'X', 'CCONJ', 'AUX'], dtype=object)

In [0]:
df_tweet.groupby(['Tag','POS']).size().reset_index(name='counts')

,Tag,POS,counts
0,ADJ,CC,1
1,ADJ,CD,1
2,ADJ,FW,22
3,ADJ,JJ,67
4,ADJ,NN,178
...,...,...,...
193,X,IN,1
194,X,JJ,3
195,X,NN,40
196,X,VB,1


In [0]:
#Checking null values, if any.
df_tweet.isnull().sum()

Sentence #    0
Word          0
POS           0
Tag           0
dtype: int64

In [0]:
df_tweet = df_tweet.fillna(method = 'ffill')

## Treinamento

In [0]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
#Displaying one full sentence
getter = sentence(df_tweet)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'@ ForeverPlayerG tem que ser , essa o áudio tava ruim . Próxima faz a farm de iron na live .'

In [0]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('esse', 'NN', 'PRON'), ('João', 'NNP', 'PROPN'), ('Guilherme', 'NNP', 'PROPN'), ('ta', 'NN', 'VERB'), ('mt', 'NN', 'NOUN'), ('estranho', 'NN', 'NOUN')]


In [0]:
sentences = getter.sentences

In [0]:
import re

def arroba(w):
  if(re.match(r'[^@]', w)):
    return True 
  else:
    return False

def hashtag(w):
  if(re.match(r'[^#]', w)):
    return True 
  else:
    return False


In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            'arroba': arroba(word1),
            'hashtag': hashtag(word1),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],

        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],

        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2postags(sent):
    return [postag for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]
  

In [0]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [0]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

In [0]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.8923532657893002


In [0]:
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         ADJ       0.72      0.41      0.52        64
         ADP       0.99      0.97      0.98       263
         ADV       0.98      0.93      0.95       313
         AUX       1.00      0.75      0.86         4
       CCONJ       1.00      1.00      1.00         6
         DET       0.99      0.95      0.97       224
        NOUN       0.80      0.88      0.84       943
         NUM       0.94      0.94      0.94        32
        PRON       0.99      0.99      0.99       154
       PROPN       0.68      0.62      0.65       196
       PUNCT       1.00      0.98      0.99       428
       SCONJ       0.99      0.96      0.98       106
        VERB       0.90      0.88      0.89       828
           X       1.00      1.00      1.00        14

    accuracy                           0.89      3575
   macro avg       0.93      0.88      0.90      3575
weighted avg       0.89      0.89      0.89      3575



# **Dataset Leis**



## Geração e Organização do Dataframe


In [0]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1TVkaoekk8xE6bGCbrVh3bn1TH9YuNPs3' -O dataset.ptbr_leis.txt

--2019-11-13 19:20:44--  https://docs.google.com/uc?export=download&id=1TVkaoekk8xE6bGCbrVh3bn1TH9YuNPs3
Resolving docs.google.com (docs.google.com)... 74.125.195.139, 74.125.195.113, 74.125.195.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dfa4g29rc78l8ncc5vkpcjmo3j7g4cpo/1573668000000/17742136402446089806/*/1TVkaoekk8xE6bGCbrVh3bn1TH9YuNPs3?e=download [following]
--2019-11-13 19:20:45--  https://doc-08-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dfa4g29rc78l8ncc5vkpcjmo3j7g4cpo/1573668000000/17742136402446089806/*/1TVkaoekk8xE6bGCbrVh3bn1TH9YuNPs3?e=download
Resolving doc-08-9o-docs.googleusercontent.com (doc-08-9o-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to doc-08-9o-docs.googleusercontent.com (doc-08-9o-do

In [0]:
f = open('dataset.ptbr_leis.txt', 'r')
new_sent = []
all_sent = [abs]
for line in f:
  if(line != '\n' and line != 'EOF' and line[0] != '<'):
    w, k = line.split(' ')
    new_sent.append(w)
  else:
    if(len(new_sent) > 0):
      all_sent.append(" ".join(new_sent))
      new_sent.clear()

df_leis = pd.DataFrame(all_sent, columns=['Sentence #'])
df_leis['Word'] = np.nan
df_leis = df_leis.iloc[1:]
df_leis.reset_index(inplace=True)
del df_leis['index']
df_leis

,Sentence #,Word
0,EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO ...,NaN
1,"- O art . 178 , II , do CPC prescreve que comp...",NaN
2,- Tratando-se de ação indenizatória ajuizada p...,NaN
3,- Tendo o vício sido arguido pelo Parquet nest...,NaN
4,- Preliminar acolhida para reconhecer a nulida...,NaN
...,...,...
7822,ISTO POSTO ACORDAM os Ministros da Segunda Tur...,NaN
7823,Verbas Rescisórias .,NaN
7824,"Controversas '' , por ofensa ao artigo 467 da ...",NaN
7825,"Brasília , 04 de novembro de 2015 .",NaN


## Tokenização

In [0]:
len(df_leis)

7827

In [0]:
print(type(df_leis))
df_leis = tokenize_df(df_leis)

In [31]:
df_leis

,Sentence #,Word,POS,Tag
0,Sentence: 0,EMENTA,NN,VERB
1,Sentence: 0,:,:,PUNCT
2,Sentence: 0,APELAÇÃO,NNP,PROPN
3,Sentence: 0,CÍVEL,NNP,ADV
4,Sentence: 0,-,:,PUNCT
...,...,...,...,...
229463,Sentence: 7826,fls,NN,NOUN
229464,Sentence: 7826,.,.,PUNCT
229465,Sentence: 7826,PROCESSO,NNP,PROPN
229466,Sentence: 7826,Nº,NNP,NOUN


In [0]:
#df_leis.to_csv("dataframe_leis.csv")

##  Treinamento



In [51]:
#Displaying one full sentence
getter = sentence(df_leis)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA .'

In [52]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('-', ':', 'PUNCT'), ('O', 'NN', 'DET'), ('art', 'NN', 'NOUN'), ('.', '.', 'PUNCT'), ('178', 'CD', 'NUM'), (',', ',', 'PUNCT'), ('II', 'NNP', 'PROPN'), (',', ',', 'PUNCT'), ('do', 'VBP', 'ADV'), ('CPC', 'NNP', 'NOUN'), ('prescreve', 'VB', 'VERB'), ('que', 'NN', 'SCONJ'), ('compete', 'JJ', 'VERB'), ('ao', 'NN', 'ADP'), ('Ministério', 'NNP', 'NOUN'), ('Público', 'NNP', 'NOUN'), ('intervir', 'NN', 'VERB'), ('nas', 'NNS', 'CCONJ'), ('causas', 'VBP', 'NOUN'), ('em', 'JJ', 'ADP'), ('que', 'NN', 'SCONJ'), ('há', 'NN', 'VERB'), ('interesses', 'VBZ', 'NOUN'), ('de', 'FW', 'ADP'), ('incapazes', 'FW', 'ADJ'), (',', ',', 'PUNCT'), ('dispondo', 'FW', 'VERB'), ('o', 'JJ', 'DET'), ('art', 'NN', 'NOUN'), ('.', '.', 'PUNCT'), ('279', 'CD', 'NUM'), ('do', 'VBP', 'ADV'), ('mesmo', 'JJR', 'ADV'), ('diploma', 'VB', 'NOUN'), ('que', 'JJ', 'SCONJ'), ('o', 'JJ', 'DET'), ('processo', 'NN', 'NOUN'), ('será', 'NN', 'VERB'), ('nulo', 'FW', 'NOUN'), ('quando', 'NN', 'ADV'), ('o', 'NN', 'DET'), ('Ministério', 'NNP

In [0]:
sentences = getter.sentences

In [0]:
def traco(w):
  if(w == '-'):
    return True
  else:
    return False
def doispontos(w):
  if(w == ':'):
    return True
  else:
    return False

def word2features1(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'traco': traco(word),
        'doispontos': doispontos(w),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            'traco': traco(word),
            'doispontos': doispontos(w),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],

        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            'traco': traco(word),
            'doispontos': doispontos(w),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],

        })
    else:
        features['EOS'] = True

    return features


def sent2features1(sent):
    return [word2features1(sent, i) for i in range(len(sent))]

def sent2labels1(sent):
    return [label for token, postag, label in sent]

def sent2postags1(sent):
    return [postag for token, postag, label in sent]

def sent2tokens1(sent):
    return [token for token, postag, label in sent]
  

In [0]:
X1 = [sent2features1(s) for s in sentences]
y1 = [sent2labels1(s) for s in sentences]

In [0]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2)

In [62]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.07,
         max_iterations = 200,
         all_possible_transitions = False)
crf.fit(X1_train, y1_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.07, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=200,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
#Predicting on the test set.
y1_pred = crf.predict(X1_test)

In [64]:
f1_score1 = flat_f1_score(y1_test, y1_pred, average = 'weighted')
print(f1_score1)

0.9840799510265604


In [65]:
report1 = flat_classification_report(y1_test, y1_pred)
print(report1)

              precision    recall  f1-score   support

         ADJ       0.98      0.96      0.97      1469
         ADP       1.00      1.00      1.00      6664
         ADV       0.99      0.99      0.99      3029
         AUX       1.00      1.00      1.00         5
       CCONJ       1.00      1.00      1.00        23
         DET       1.00      1.00      1.00      2202
        NOUN       0.97      0.98      0.98     11669
         NUM       0.97      0.96      0.96      1238
        PRON       1.00      1.00      1.00       601
       PROPN       0.96      0.93      0.94      2779
       PUNCT       1.00      1.00      1.00      8364
       SCONJ       1.00      1.00      1.00      1056
        VERB       0.98      0.98      0.98      6354
           X       1.00      0.89      0.94        27

    accuracy                           0.98     45480
   macro avg       0.99      0.98      0.98     45480
weighted avg       0.98      0.98      0.98     45480

